In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
from pathlib import Path

import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from skimage.transform import resize

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import models

In [ ]:
BASE_PATH = Path("..")
sys.path.append(str(BASE_PATH.resolve()))

In [ ]:
from mobilenetv3 import MobilenetV3

In [ ]:
tf.__version__

In [ ]:
from tensorflow.keras.applications import MobileNetV3Small

## CIFAR-10 Dataset

In [ ]:
CLASS_NAMES = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

In [ ]:
cifar10 = tf.keras.datasets.cifar10

(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

In [ ]:
print(train_images.shape, train_labels.shape)

In [ ]:
# train_images = np.expand_dims(train_images, axis = 3)
# test_images = np.expand_dims(test_images, axis = 3)

# train_images = resize(train_images, (len(train_images),224,224,1))
# test_images = resize(test_images, (len(test_images),224,224,1))

train_images = train_images / 255.
test_images = test_images / 255.

train_labels_onehot = tf.keras.utils.to_categorical(train_labels, num_classes=len(CLASS_NAMES))
test_labels_onehot = tf.keras.utils.to_categorical(test_labels, num_classes=len(CLASS_NAMES))

In [ ]:
train_images.shape

In [ ]:
SAMPLE_SIZE = 10
samples = np.random.randint(0, len(train_images), size = SAMPLE_SIZE)

fig, axarr = plt.subplots(5, SAMPLE_SIZE//5, figsize = (10,10))
axs = axarr.ravel()

for idx, sample in enumerate(samples):
    axs[idx].imshow(train_images[sample,:,:,:])
    axs[idx].set_title(CLASS_NAMES[train_labels[sample,0]])
    
plt.tight_layout()

## Test MobilenetV3

In [ ]:
mobilenetv3 = MobilenetV3(train_images.shape[1:], len(CLASS_NAMES), alpha=1.0, size = 'small')
model = mobilenetv3()


In [ ]:
# model = MobileNetV3Small(input_shape = train_images.shape[1:], classes=len(CLASS_NAMES), include_top=True, weights = None)


In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.0003)
model.compile(optimizer=opt,
          loss=['categorical_crossentropy'],
          metrics=['accuracy'])
    

In [ ]:
tf.keras.utils.plot_model(model, "mobilenet_v3_small.png", show_shapes=True)


In [ ]:
model.summary()

In [ ]:
model.save('mobilenetv3_small.h5')

In [ ]:
model.fit(x = train_images, y = train_labels_onehot, validation_split=0.20, verbose=1,epochs = 10)

## Evaluate MobilenetV3

In [ ]:
test_loss, test_acc = model.evaluate(test_images,  test_labels_onehot, verbose=2)

print('\nTest accuracy:', test_acc)